# BitChute Scraper Examples

In [ ]:
import bitchute as bc

In [ ]:
# Initialize the BitChute API client
api = bc.BitChuteAPI(verbose=False)

## Get Platform Recommendations

In [ ]:
# Retrieve videos listed under "Trending"
trending = api.get_trending_videos('day', include_details=False, limit=2)
print(f"Success! Got {len(trending)} videos")
print()
trending.head(2)

In [ ]:
# Retrieve videos listed under "Fresh"
popular = api.get_popular_videos(include_details=False)
print(f"Success! Got {len(popular)} videos")
print()
popular.head(2)

In [ ]:
# Retrieve videos listed under "Shorts"
short = api.get_short_videos()
print(f"Success! Got {len(short)} videos")
print()
short.head(2)

In [ ]:
# Retrieve videos listed under "Member Picked"
# CURRENTLY A POTENTIALLY INTERESTNIG DATAPOINT IS NOT INCLUDED IN THE RESULTS: date_liked. NEEDS ENHANCEMENT

member_picked = api.get_member_picked_videos(include_details=False)
print(f"Success! Got {len(member_picked)} videos")
print()
member_picked.head(2)

In [ ]:
# Retrieve videos listed under "all"
all_videos = api.get_all_videos(include_details=False)
print(f"Success! Got {len(all_videos)} videos")
print()
all_videos.head(2)

In [ ]:
# Retrieve trending hashtags
hashtags = api.get_trending_hashtags()
print(f"Success! Got {len(hashtags)} hashtags")
print()
hashtags.head(2)

## Search the BitChute Platform

In [ ]:
# Search Videos by Query
#
# Options for `sensitivity` and `sort`:
# sensitivity: 'nsfw', 'nsfl' defaults to 'normal'
# sort: 'views', 'old' defaults to 'new'

search = api.search_videos('bitcoin', limit=10, sensitivity='normal', sort='new', include_details=True) 
print(f"Success! Got {len(search)} videos")
print()
search.head(2)

In [ ]:
# Retrieve Videos by Hashtag
videos = api.get_videos_by_hashtag('truth', limit=10, include_details=True)
videos

In [ ]:
# Search Channels by Query
#
# Options for `sensitivity`:
# sensitivity: 'nsfw', 'nsfl' defaults to 'normal'

channels = api.search_channels('truth', limit=10, sensitivity='normal', include_details=True)
print(f"Success! Got {len(channels)} channels")
print()
channels.head(2)

## Retrieve data for individual videos and channels
### Video

In [ ]:
video = api.get_video_info('JO_qKQUkTrg')
type(video)

### Channel

In [ ]:
# Get channel information:
# Get channel details
channel = api.get_channel_info('UnweMhJQFhlW')
type(channel)

In [ ]:
# Get channel videos
videos = api.get_channel_videos('UnweMhJQFhlW', limit=50000, include_details=True)
print(f"Success! Got {len(videos)} videos")
print()
videos.head(2)

## Downloading Thumbnails and Videos Example

In [ ]:
api = bc.BitChuteAPI(
        enable_downloads=True,
        thumbnail_folder="thumbnails",
        video_folder="videos",
        force_redownload=False,
        max_concurrent_downloads=3,
        verbose=True
    )

# CAREFUL: This will download the latest videos from the channel, which can be a lot of data.
videos = api.get_recent_videos(limit=4, include_details=True, download_thumbnails=True, download_videos=True)